In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_news = load_dataset("ag_news")
dataset_imdb = load_dataset('imdb')
print((dataset_imdb))

In [ ]:
import random

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short_news = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 2000:
    dataset_short_news.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1

random.shuffle(dataset_short_news)
dataset_news = {'train': dataset_short_news[:6400], 'test': dataset_short_news[6400:]}

In [ ]:
train_texts_news = [s['news'] for s in dataset_news['train']]
test_texts_news = [s['news'] for s in dataset_news['test']]

train_labels_news = [s['label'] for s in dataset_news['train']]
test_labels_news = [s['label'] for s in dataset_news['test']]

# Выводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

In [ ]:
count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 3000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 3000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1

random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:4800], 'test': dataset_short_imdb[4800:]}

train_texts_imdb = [s['text'] for s in dataset_imdb['train']]
test_texts_imdb = [s['text'] for s in dataset_imdb['test']]

train_labels_imdb = [s['label'] for s in dataset_imdb['train']]
test_labels_imdb = [s['label'] for s in dataset_imdb['test']]

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words("english")))

def preprocess(text):
  tokens = word_tokenize(text)
  tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens]
  tokens = [word for word in tokens if word not in stop_words]
  tokens = [word.lower() for word in tokens]
  return ' '.join(tokens)

In [ ]:
train_texts_news = [preprocess(i) for i in train_texts_news]

In [ ]:
train_texts_imdb = [preprocess(i) for i in train_texts_imdb]

# Обучаем XGBoost - GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import pandas as pd

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', GradientBoostingClassifier())
])

In [ ]:
param_grid_GBM_news = {
    'clf__max_depth': [3, 4, 5],
    'clf__learning_rate': [0.01, 0.1, 1],
    'clf__subsample': [0.7, 0.9, 1],
    'clf__max_features':['sqrt', 0.5]
}

In [ ]:
res_news_GBM = []

for m_d in param_grid_GBM_news['clf__max_depth']:
  for lr in param_grid_GBM_news['clf__learning_rate']:
    for sub in param_grid_GBM_news['clf__subsample']:
      for m_f in param_grid_GBM_news['clf__max_features']:
        pipeline.set_params(
          clf__max_depth = m_d,
          clf__learning_rate = lr,
          clf__subsample = sub,
          clf__max_features = m_f
        )
        pipeline.fit(train_texts_news, train_labels_news)
        y_pred_news = pipeline.predict(test_texts_news)

        f1 = f1_score(test_labels_news, y_pred_news, average='weighted')
        current = [m_d, lr, sub, m_f, f1]
        res_news_GBM.append(current)
        if len(res_news_GBM) % 10 == 0:
          print(len(res_news_GBM))

In [ ]:
df_results = pd.DataFrame(res_news_GBM, columns=['max_depth', 'learning_rate', 'subsample', 'max_features', 'f1-score'])
df_results.to_excel("results_news_GBM.xlsx", index=False)

In [ ]:
pipeline_imdb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', GradientBoostingClassifier())
])

In [ ]:
param_grid_GBM_imdb = {
    'clf__max_depth': [3, 5, 7],
    'clf__learning_rate': [0.01, 0.1, 1],
    'clf__subsample': [0.6, 0.8, 0.9],
    'clf__max_features':['sqrt', 0.1]
}

In [ ]:
res_imdb_GBM = []

for m_d in param_grid_GBM_imdb['clf__max_depth']:
  for lr in param_grid_GBM_imdb['clf__learning_rate']:
    for sub in param_grid_GBM_imdb['clf__subsample']:
      for m_f in param_grid_GBM_imdb['clf__max_features']:
        pipeline_imdb.set_params(
          clf__max_depth = m_d,
          clf__learning_rate = lr,
          clf__subsample = sub,
          clf__max_features = m_f
        )
        pipeline_imdb.fit(train_texts_imdb, train_labels_imdb)
        y_pred_imdb = pipeline_imdb.predict(test_texts_imdb)

        f1 = f1_score(test_labels_imdb, y_pred_imdb, average='weighted')
        current = [m_d, lr, sub, m_f, f1]
        res_imdb_GBM.append(current)
        if len(res_imdb_GBM) % 10 == 0:
          print(len(res_imdb_GBM))

In [ ]:
df_results = pd.DataFrame(res_imdb_GBM, columns=['max_depth', 'learning_rate', 'subsample', 'max_features', 'f1-score'])
df_results.to_excel("results_imdb_GBM.xlsx", index=False)

# Обучаем AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
base_est = DecisionTreeClassifier(max_depth=3)

ada = AdaBoostClassifier(
    estimator=base_est,
    n_estimators=50,
    learning_rate=0.1,
    random_state=42
)

# 3. Создаем pipeline
pipeline_news_ada = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ada)
])

In [ ]:
param_news_ada = {
    'clf__estimator__max_depth': [1, 3, 2],
    'clf__n_estimators': [50, 100, 200],
    'clf__learning_rate': [0.01, 0.1, 1.0]
}

In [ ]:
res_news_ada = []

for m_d in param_news_ada['clf__estimator__max_depth']:
  for est in param_news_ada['clf__n_estimators']:
    for lr in param_news_ada['clf__learning_rate']:
      pipeline_news_ada.set_params(
          clf__estimator__max_depth = m_d,
          clf__n_estimators = est,
          clf__learning_rate = lr
      )
      pipeline_news_ada.fit(train_texts_news, train_labels_news)
      y_pred_news = pipeline_news_ada.predict(test_texts_news)

      f1 = f1_score(test_labels_news, y_pred_news, average='weighted')
      current = [m_d, est, lr, f1]
      res_news_ada.append(current)
      if len(res_news_ada) % 10 == 0:
        print(len(res_news_ada))

In [ ]:
df_results = pd.DataFrame(res_news_ada, columns=['max_depth', 'n_estimators', 'learning_rate', 'f1-score'])
df_results.to_excel("results_news_ada.xlsx", index=False)

In [ ]:
base_est = DecisionTreeClassifier(max_depth=3)

ada = AdaBoostClassifier(
    estimator=base_est,
    n_estimators=50,
    learning_rate=0.1,
    random_state=42
)

# 3. Создаем pipeline
pipeline_imdb_ada = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', ada)
])

In [ ]:
param_imdb_ada = {
    'clf__estimator__max_depth': [1, 3, 5],
    'clf__n_estimators': [50, 200, 300],
    'clf__learning_rate': [0.01, 0.1, 1.0]
}

In [ ]:
res_imdb_ada = []

for m_d in param_imdb_ada['clf__estimator__max_depth']:
  for est in param_imdb_ada['clf__n_estimators']:
    for lr in param_imdb_ada['clf__learning_rate']:
      pipeline_imdb_ada.set_params(
          clf__estimator__max_depth = m_d,
          clf__n_estimators = est,
          clf__learning_rate = lr
      )
      pipeline_imdb_ada.fit(train_texts_imdb, train_labels_imdb)
      y_pred_imdb = pipeline_imdb_ada.predict(test_texts_imdb)

      f1 = f1_score(test_labels_imdb, y_pred_imdb, average='weighted')
      current = [m_d, est, lr, f1]
      res_imdb_ada.append(current)
      if len(res_imdb_ada) % 10 == 0:
        print(len(res_imdb_ada))

In [ ]:
df_results = pd.DataFrame(res_imdb_ada, columns=['max_depth', 'n_estimators', 'learning_rate', 'f1-score'])
df_results.to_excel("results_imdb_ada.xlsx", index=False)

# Обучаем XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
pipeline_news_xgboost = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier())
])

In [ ]:
param_news_xgboost = {
    'clf__max_depth': [3, 5, 7],
    'clf__colsample_bytree': [0.7, 1],
    'clf__subsample': [0.7, 1],
    'clf__reg_lambda': [0.1, 0.5, 1],
    'clf__learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
res_news_xg = []
for md in param_news_xgboost['clf__max_depth']:
  for col in param_news_xgboost['clf__colsample_bytree']:
    for sub in param_news_xgboost['clf__subsample']:
      for reg in param_news_xgboost['clf__reg_lambda']:
        for lr in param_news_xgboost['clf__learning_rate']:
          pipeline_news_xgboost.set_params(
            clf__max_depth = md,
            clf__colsample_bytree = col,
            clf__subsample = sub,
            clf__reg_lambda = reg,
            clf__learning_rate = lr
          )
          pipeline_news_xgboost.fit(train_texts_news, train_labels_news)
          y_pred_news = pipeline_news_xgboost.predict(test_texts_news)
          f1 = f1_score(test_labels_news, y_pred_news, average='weighted')
          current = [md, col, sub, reg, lr, f1]
          res_news_xg.append(current)
          if len(res_news_xg) % 10 == 0:
            print(len(res_news_xg))

In [ ]:
df_results = pd.DataFrame(res_news_xg, columns=['max_depth', 'colsample_bytree', 'subsample','reg_lambda','learning_rate', 'f1-score'])
df_results.to_excel("results_news_xg.xlsx", index=False)

In [ ]:
pipeline_imdb_xgboost = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier())
])

In [ ]:
param_imdb_xgboost = {
    'clf__max_depth': [3, 5, 8],
    'clf__colsample_bytree': [0.5, 1],
    'clf__subsample': [0.6, 0.9],
    'clf__reg_lambda': [0.1, 0.5, 1],
    'clf__learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
res_imdb_xg = []

for md in param_imdb_xgboost['clf__max_depth']:
  for col in param_imdb_xgboost['clf__colsample_bytree']:
    for sub in param_imdb_xgboost['clf__subsample']:
      for reg in param_imdb_xgboost['clf__reg_lambda']:
        for lr in param_imdb_xgboost['clf__learning_rate']:
          pipeline_imdb_xgboost.set_params(
            clf__max_depth = md,
            clf__colsample_bytree = col,
            clf__subsample = sub,
            clf__reg_lambda = reg,
            clf__learning_rate = lr
          )
          pipeline_imdb_xgboost.fit(train_texts_imdb, train_labels_imdb)
          y_pred_imdb = pipeline_imdb_xgboost.predict(test_texts_imdb)
          f1 = f1_score(test_labels_imdb, y_pred_imdb, average='weighted')
          current = [md, col, sub, reg, lr, f1]
          res_imdb_xg.append(current)
          if len(res_imdb_xg) % 10 == 0:
            print(len(res_imdb_xg))

In [ ]:
df_results = pd.DataFrame(res_imdb_xg, columns=['max_depth', 'colsample_bytree', 'subsample','reg_lambda','learning_rate', 'f1-score'])
df_results.to_excel("results_imdb_xg.xlsx", index=False)